This notebook heavily uses the code from https://github.com/felipemaiapolo/cit.git to reproduce their results.
It assumes the simulated null experiment (`run_h0_car_data.py`) results are saved to `./car-final.pt` and that the p-value experiment results are provided here (`run_pval_car_data.py` simply prints those results).

It's better to run this notebook in colab due to extra dependencies + the need to run RBPT locally. Overall, running the notebook should take well under an hour to run.

In [ ]:
!git clone https://github.com/felipemaiapolo/cit.git
%cd ./cit
# !pip install -r requirements.txt
!pip install catboost  # the only one missing from colab

In [ ]:
from general import *
import time
import copy
import multiprocessing as mp
import seaborn as sns

cpu=mp.cpu_count()
cpu

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
import copy
import time
from general import *
from exp1 import get_pval_stfr, get_pval_crt

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


###Regression
def get_pval_gcm(X, Z, Y, g2, p_model):
    n = X.shape[0]
    rx = X-p_model.predict_proba(Z)[:,1].reshape(X.shape)
    ry = Y-g2.predict(None, Z)
    T = rx.squeeze()*ry.squeeze()
    pval = 2*(1 - scipy.stats.norm.cdf(abs(np.sqrt(n)*np.mean(T)/np.std(T))))
    return pval

def get_pval_rbpt(X, Z, Y, H, g1, loss='mse'):
    n = X.shape[0]
    XZ = np.hstack((X, Z))
    loss1 = get_loss(Y, g1.predict(X,Z).reshape((-1,1)), loss=loss)
    loss2 = get_loss(Y, H.reshape((-1,1)), loss=loss)
    T = loss2-loss1
    pval = 1 - scipy.stats.norm.cdf(np.sqrt(n)*np.mean(T)/np.std(T))
    return pval

def get_pval_rbpt2(X, Z, Y, g1, h, loss='mae'):
    n = X.shape[0]
    XZ = np.hstack((X, Z))
    loss1 = get_loss(Y, g1.predict(X,Z).reshape((-1,1)), loss=loss)
    loss2 = get_loss(Y, h.predict(Z).reshape((-1,1)), loss=loss)
    T = loss2-loss1
    pval = 1 - scipy.stats.norm.cdf(np.sqrt(n)*np.mean(T)/np.std(T))
    return pval


# Our addition:
def get_pval_rbpt2_ub(X, Z, Y, g1, h, loss='mse'):
    assert loss == 'mse'
    n = X.shape[0]
    XZ = np.hstack((X, Z))
    g_pred = g1.predict(X,Z).reshape((-1,1))
    h_pred = h.predict(Z).reshape((-1,1))
    loss1 = get_loss(Y, g_pred, loss=loss)
    loss2 = get_loss(Y, h_pred, loss=loss)
    bias_correction = get_loss(g_pred, h_pred, loss=loss)
    #!!!!!!!!!
    T = loss2-loss1 + bias_correction
    pval = 1 - scipy.stats.norm.cdf(np.sqrt(n)*np.mean(T)/np.std(T))
    return pval


def get_h(X, y, validation_split=.1, verbose=False, random_state=None):

    ### Paramaters
    early_stopping_rounds=10
    loss='MultiRMSE'

    ### Validating
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=random_state)

    m = CatBoostRegressor(loss_function = loss,
                          eval_metric = loss,
                          thread_count=-1,
                          random_seed=random_state)

    m.fit(X_train, y_train, verbose=verbose,
          eval_set=(X_val, y_val),
          early_stopping_rounds = early_stopping_rounds)


    ### Final model
    m2 = CatBoostRegressor(iterations=int(m.tree_count_),
                           loss_function = loss,
                           eval_metric = loss,
                           thread_count=-1,
                           random_seed=random_state)

    m2.fit(X, y, verbose=verbose)

    return m2


#####
def exp2(it, n_vals, loss, alpha, B):

    states=['ca','il','mo','tx']
    pvals=[]
    times=[]
    count=0

    for s in states:

        data = pd.read_csv('data/car-insurance-public/data/' + s + '-per-zip.csv')
        companies = list(set(data.companies_name))

        for cia in companies:

            data = pd.read_csv('data/car-insurance-public/data/' + s + '-per-zip.csv')
            data = data.loc[:,['state_risk','combined_premium','minority','companies_name']].dropna()
            data = data.loc[data.companies_name == cia]

            Z = np.array(data.state_risk).reshape((-1,1))
            # print(Z.shape)
            # continue
            Y = np.array(data.combined_premium).reshape((-1,1))
            X = (1*np.array(data.minority)).reshape((-1,1))

            #bins = np.percentile(Z, np.linspace(0,100,n_vals+2))
            bins = np.linspace(np.min(Z),np.max(Z),n_vals+2)
            bins = bins[1:-1]
            Y_ci = copy.deepcopy(Y)
            Z_bin = np.array([find_nearest(bins, z) for z in Z.squeeze()]).reshape(Z.shape)

            for val in np.unique(Z_bin):
                ind = Z_bin==val
                rng = np.random.RandomState(it)
                ind2 = rng.choice(np.sum(ind),np.sum(ind),replace=False)
                Y_ci[ind] = Y_ci[ind][ind2]

            X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(X, Y_ci, Z_bin, test_size=.3, random_state=it)

            ###Fitting models
            g1 = g()
            g1.fit(X_train, Z_train, Y_train)
            g2 = g()
            g2.fit(None, Z_train, Y_train)

            ###RBPT
            start_time = time.time()
            p = LogisticRegressionCV(cv=5, scoring='neg_log_loss', solver='liblinear', random_state=0).fit(Z_train, X_train.squeeze())
            H_test = np.sum(p.predict_proba(Z_test)*np.hstack((g1.predict(np.zeros(X_test.shape),Z_test).reshape(-1,1),
                                                               g1.predict(np.ones(X_test.shape),Z_test).reshape(-1,1))), axis=1).reshape(-1,1)
            pval_rbpt = get_pval_rbpt(X_test, Z_test, Y_test, H_test, g1, loss=loss)
            time_rbpt = time.time() - start_time

            ###RBPT2
            start_time = time.time()
            h = get_h(Z_train, g1.predict(X_train,Z_train).squeeze())
            pval_rbpt2 = get_pval_rbpt2(X_test, Z_test, Y_test, g1, h, loss=loss)
            time_rbpt2 = time.time() - start_time


            ###RBPT2 unbiased
            start_time = time.time()
            pval_rbpt2_ub = get_pval_rbpt2_ub(X_test, Z_test, Y_test, g1, h, loss=loss)
            time_rbpt2_ub = time.time() - start_time


            ###GCM
            start_time = time.time()
            pval_gcm = get_pval_gcm(X_test, Z_test, Y_test, g2, p)
            time_gcm = time.time() - start_time


            ###Storing
            times.append([count, time_rbpt, time_rbpt2, time_rbpt2_ub, time_gcm])
            pvals.append([count, pval_rbpt, pval_rbpt2, pval_rbpt2_ub, pval_gcm])

        count+=1

    pvals = np.array(pvals)
    return pvals, times

# Simulated null

In [ ]:
labels = ['California','Illinois','Missouri','Texas']
alpha = 0.05
loss = 'mse'
B = 100
n_vals = 20
iterations = 50 #48

pool = mp.Pool(cpu)
out = pool.starmap(exp2, [(it, n_vals, loss, alpha, B) for it in range(iterations)])
pool.close()

RBPT paper results done, now loading the kernel-based measures:

In [ ]:
res = torch.load('car-final.pt')
res_kernels = np.zeros((4, 50, 3))

for i in range(50):
    for st_idx, state in enumerate(['ca', 'il', 'mo', 'tx']):
        for al_idx, alg in enumerate(['kci', 'kci_xsplit', 'circe']):
            res_kernels[st_idx, i, al_idx] = (np.array(res[i][state][alg])[:, 1] < alpha).mean()

Plotting everything together:

In [ ]:
# out.shape = (50, 98, 5), so 50 runs, pvals/times, 98 runs with grouping, last dim [0] is count of state
rejection_rate = np.zeros((4, out.shape[0], out.shape[-1] - 1))
alpha = 0.05
# essentially we define the acceptance rate as the average acceptance rate across companies, and estimate that.

for state_idx in range(4):
    state_vals = out[:, out[0, :, 0] == state_idx, 1:] < alpha
    rejection_rate[state_idx] = state_vals.mean(axis=1)
    
rejection_rate = np.concatenate((rejection_rate, res_kernels[:, :, :]), axis=-1) # with circe

In [ ]:
rejection_rate_for_pdf = np.zeros((3, np.prod(rejection_rate.shape)))
labels = np.zeros_like(rejection_rate)
for state_idx in range(4):
    for algorithm in range(rejection_rate.shape[-1]):
        l = rejection_rate.shape[1] * (algorithm + state_idx * rejection_rate.shape[-1])
        rejection_rate_for_pdf[0, l:l+rejection_rate.shape[1]] = state_idx
        rejection_rate_for_pdf[1, l:l+rejection_rate.shape[1]] = algorithm
        rejection_rate_for_pdf[2, l:l+rejection_rate.shape[1]] = rejection_rate[state_idx, :, algorithm]
        
df = pd.DataFrame(rejection_rate_for_pdf.T)

for idx, state_name in enumerate(['California','Illinois','Missouri','Texas']):
    df.loc[df[0] == idx, 0] = state_name

for idx, alg_name in enumerate(['RBPT','RBPT2','RBPT2\'','GCM', 'KCI', 'SplitKCI', 'CIRCE']): #
    df.loc[df[1] == idx, 1] = alg_name

df.rename(columns={0: 'State', 1: 'Algorithm', 2: 'Type I error'}, inplace=True)

grid = sns.catplot(
    df, kind="bar",
    x='Algorithm', y='Type I error', col='State',
    height=7, aspect=.6,
    errorbar='se', alpha=0.5
)

axes = grid.axes.flatten()

# iterate through the axes
for i, ax in enumerate(axes):
    ax.axhline(alpha, ls='--', c='#5F5F5F')

plt.savefig('./figs/data_h0_catplot_with_circe.pdf')

plt.show()

# p-values

In [ ]:
states=['ca','il','mo','tx']
labels = ['California','Illinois','Missouri','Texas']
alpha = 0.05
loss = 'mse'
B = 100

data = pd.read_csv('data/car-insurance-public/data/mo-per-zip.csv')
data.head()

random_state=42
import torch
# np.random.seed(random_state)

In [ ]:
pvals = []
times = []

for s in tqdm(states):
    data = pd.read_csv('data/car-insurance-public/data/' + s + '-per-zip.csv')

    Z = np.array(data.state_risk).reshape((-1,1))
    Y = np.array(data.combined_premium).reshape((-1,1))
    X = (1*np.array(data.minority)).reshape((-1,1))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3, random_state=random_state)
    Z_train, Z_test, _, _ = train_test_split(Z, Y, test_size=.3, random_state=random_state)

    ###Fitting models
    g1 = g()
    g1.fit(X_train, Z_train, Y_train)
    g2 = g()
    g2.fit(None, Z_train, Y_train)

    ###RBPT
    p = LogisticRegressionCV(cv=5, scoring='neg_log_loss', solver='liblinear', random_state=0).fit(Z_train, X_train.squeeze())
    H_test = np.sum(p.predict_proba(Z_test)*np.hstack((g1.predict(np.zeros(X_test.shape),Z_test).reshape(-1,1),
                                                       g1.predict(np.ones(X_test.shape),Z_test).reshape(-1,1))), axis=1).reshape(-1,1)
    pval_rbpt = get_pval_rbpt(X_test, Z_test, Y_test, H_test, g1, loss=loss)

    ###RBPT2
    h = get_h(Z_train, g1.predict(X_train,Z_train).squeeze())
    pval_rbpt2 = get_pval_rbpt2(X_test, Z_test, Y_test, g1, h, loss=loss)

    ###RBPT2 unbiased
    start_time = time.time()
    pval_rbpt2_ub = get_pval_rbpt2_ub(X_test, Z_test, Y_test, g1, h, loss=loss)
    time_rbpt2_ub = time.time() - start_time
    
    ###GCM
    pval_gcm = get_pval_gcm(X_test, Z_test, Y_test, g2, p)


    ###Storing
    # pvals.append([pval_rbpt, pval_rbpt2, pval_stfr, pval_gcm, pval_crt, pval_cpt])
    pvals.append([pval_rbpt, pval_rbpt2, pval_rbpt2_ub, pval_gcm])

pvals=np.array(pvals)
print(pvals)

Copied from 'run_pval_car_data.py' output:

In [ ]:
# circe:
circe_res = np.array([0.003, 0., 0.003, 0.002])

# kci
kci_res = np.array([0.016, 0.,    0.,    0.,   ])

# split-kci
split_kci_res = np.array([0.027, 0.,    0.,    0.,   ])

In [ ]:
pvals_new = np.hstack((pvals, kci_res[:, None], split_kci_res[:, None], circe_res[:, None]))
pvals_old = pvals.copy()
pvals = pvals_new

In [ ]:
res = np.zeros((3, np.prod(pvals.shape)))

for state_idx in range(4):
    for alg_idx in range(pvals.shape[1]):
        res[0, alg_idx + state_idx * pvals.shape[1]] = state_idx
        res[1, alg_idx + state_idx * pvals.shape[1]] = alg_idx
        res[2, alg_idx + state_idx * pvals.shape[1]] = pvals[state_idx, alg_idx]

In [ ]:
df = pd.DataFrame(res.T)

for idx, state_name in enumerate(['California','Illinois','Missouri','Texas']):
    df.loc[df[0] == idx, 0] = state_name

for idx, alg_name in enumerate(['RBPT','RBPT2','RBPT2\'','GCM', 'KCI', 'SplitKCI', 'CIRCE']):
    df.loc[df[1] == idx, 1] = alg_name

df.rename(columns={0: 'State', 1: 'Algorithm', 2: 'p-value'}, inplace=True)

grid = sns.catplot(
    df, kind="bar",
    x='Algorithm', y='p-value', col='State',
    height=7, aspect=.6,
    errorbar='se', alpha=0.5
)

axes = grid.axes.flatten()

# iterate through the axes
for i, ax in enumerate(axes):
    ax.axhline(alpha, ls='--', c='#5F5F5F')

plt.savefig('./figs/data_pval_catplot_circe.pdf')

plt.show()